In [1]:
GLOBAL_SEED = 7532

from numpy.random import seed
seed(GLOBAL_SEED)
from tensorflow import set_random_seed
set_random_seed(GLOBAL_SEED)

import numpy as np
import pandas as pd

from keras.models import Model, load_model, Sequential
from keras.callbacks import Callback, EarlyStopping, History, ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam
from keras.layers import BatchNormalization, Dense, Dropout, Conv2D, Flatten, Input, MaxPool2D
from keras.preprocessing.image import ImageDataGenerator

from sklearn.utils.class_weight import compute_class_weight

Using TensorFlow backend.


Read in the data.

In [2]:
train_set = pd.read_csv('train_set_metadata_HMGD.csv')
valid_set = pd.read_csv('valid_set_metadata_HMGD.csv')
test_set = pd.read_csv('test_set_metadata_HMGD.csv')

Initialize constants.

In [3]:
BEST_MODEL_PATH = 'best_model.hdf5'

INPUT_SHAPE = (320, 320, 3)

LEARNING_RATE = 0.0002
N_EPOCHS = 50
BATCH_SIZE = 32

Define the CNN model architecture.

In [4]:
def create_model(input_shape):
    model = Sequential()
    
    model.add(Conv2D(32, kernel_size=3, activation='relu',padding='same', 
                     input_shape=input_shape))
    model.add(MaxPool2D(2))
    model.add(BatchNormalization())

    model.add(Conv2D(48, kernel_size=3, activation='relu', padding='same'))
    model.add(MaxPool2D(2))
    model.add(BatchNormalization())  
    
    model.add(Conv2D(64, kernel_size=3, activation='relu', padding='same'))
    model.add(MaxPool2D(2))
    model.add(BatchNormalization())
 
    model.add(Conv2D(96, kernel_size=3, activation='relu', padding='same'))
    model.add(MaxPool2D(2))
    model.add(BatchNormalization())
    
    model.add(Conv2D(128, kernel_size=3, activation='relu', padding='same'))
    model.add(MaxPool2D(2))
    model.add(BatchNormalization())

    model.add(Conv2D(256, kernel_size=3, activation='relu', padding='same'))
    model.add(MaxPool2D(2))
    model.add(BatchNormalization())
    
    model.add(Flatten())
    model.add(Dropout(rate=0.5, seed=GLOBAL_SEED))
    model.add(Dense(units=512, activation='relu'))
    model.add(Dropout(rate=0.5, seed=GLOBAL_SEED))
    model.add(Dense(units=256, activation='relu'))
    model.add(Dropout(rate=0.5, seed=GLOBAL_SEED))
    model.add(Dense(1, activation='sigmoid'))
    
    return model

In [5]:
model = create_model(INPUT_SHAPE)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 320, 320, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 160, 160, 32)      0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 160, 160, 32)      128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 160, 160, 48)      13872     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 80, 80, 48)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 80, 80, 48)        192       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 80, 80, 64)        27712     
__________

Compute the class weights to be used during model training in order to mitigate the class imbalances.

In [6]:
class_values = train_set['gender'].values
classes = np.unique(class_values)

weights = compute_class_weight(
    'balanced', 
    classes, 
    class_values
)
class_weights = dict(zip(classes, weights))
class_weights

{0: 1.0949726587842759, 1: 0.9201873755187876}

Train the model.

In [7]:
# Create and compile the model
adam = Adam(
    lr=LEARNING_RATE
)
model.compile(
    optimizer=adam, 
    loss='binary_crossentropy',
    metrics=['binary_accuracy']
)

# Initialize callbacks
checkpoint = ModelCheckpoint(
    'model_custom_small_epoch_{epoch:03d}.hdf5', 
    monitor='val_loss', 
    save_best_only=False, 
    save_weights_only=False
)
lr_reduction = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.5, 
    patience=4
)
early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=10
)    
callback_list = [checkpoint, lr_reduction, early_stopping]

# generate and augment training and validation data
data_generator = ImageDataGenerator(
    rotation_range=20, 
    width_shift_range=0.2, 
    height_shift_range=0.2, 
    horizontal_flip=True  
)
train_data_generator = data_generator.flow_from_dataframe(
    dataframe=train_set, 
    directory='imdb_crop/_all_photos/',
    x_col='photo_path',
    y_col='gender',
    target_size=(320, 320),
    class_mode='other',
    batch_size=BATCH_SIZE,
    seed=GLOBAL_SEED
)
valid_data_generator = data_generator.flow_from_dataframe(
    dataframe=valid_set, 
    directory='imdb_crop/_all_photos/',
    x_col='photo_path',
    y_col='gender',
    target_size=(320, 320),
    class_mode='other',
    batch_size=BATCH_SIZE,
    seed=GLOBAL_SEED
)

train_steps_per_epoch = train_data_generator.n//train_data_generator.batch_size
valid_steps_per_epoch = valid_data_generator.n//valid_data_generator.batch_size

#fit the model
history = model.fit_generator(
    generator=train_data_generator,
    steps_per_epoch=train_steps_per_epoch, 
    epochs=N_EPOCHS,
    callbacks=callback_list,
    validation_data=valid_data_generator,
    validation_steps=valid_steps_per_epoch,
    class_weight=class_weights,
    workers=4, 
    verbose=1
)

Found 165400 images.
Found 4176 images.
Epoch 1/50
5168/5168 [==============================] - 2795s 541ms/step - loss: 0.5657 - binary_accuracy: 0.7240 - val_loss: 0.4199 - val_binary_accuracy: 0.8123
Epoch 2/50
5168/5168 [==============================] - 2695s 521ms/step - loss: 0.3561 - binary_accuracy: 0.8499 - val_loss: 0.3560 - val_binary_accuracy: 0.8475
Epoch 3/50
5168/5168 [==============================] - 2703s 523ms/step - loss: 0.3155 - binary_accuracy: 0.8741 - val_loss: 0.3066 - val_binary_accuracy: 0.8736
Epoch 4/50
5168/5168 [==============================] - 2693s 521ms/step - loss: 0.2992 - binary_accuracy: 0.8821 - val_loss: 0.3215 - val_binary_accuracy: 0.8733
Epoch 5/50
5168/5168 [==============================] - 2693s 521ms/step - loss: 0.2903 - binary_accuracy: 0.8865 - val_loss: 0.3057 - val_binary_accuracy: 0.8728
Epoch 6/50
5168/5168 [==============================] - 2700s 523ms/step - loss: 0.2851 - binary_accuracy: 0.8892 - val_loss: 0.3139 - val_binary

Evaluate the model on the test set.

In [9]:
evaluation_data_generator = ImageDataGenerator().flow_from_dataframe(
    dataframe=test_set, 
    directory='imdb_crop/_all_photos/',
    x_col='photo_path',
    y_col='gender',
    target_size=(320, 320),
    class_mode='other',
    batch_size=1,
    seed=GLOBAL_SEED,
    shuffle=False
)

best_model = load_model('model_custom_small_epoch_019.hdf5')
best_model.evaluate_generator(generator=evaluation_data_generator, steps=len(test_set))

Found 7621 images.


[0.28291906248427073, 0.889253378821677]